# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

def get_clf_eval(y_test, y_pred_proba, threshold=0.5):
    # 확률을 기준으로 예측 레이블 생성
    y_pred = (y_pred_proba >= threshold).astype(int)  # 0.5 이상의 확률을 양성으로 간주

    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print("Confusion Matrix:\n", confusion)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

### 데이터 읽어오기


In [9]:
RANDOM_STATE = 110

# csv 불러오기
train_data = pd.read_csv('../../data/train_data_0817.csv')
test_data  = pd.read_csv('../../data/test_data_0817.csv')

---

## 데이터 분할

In [10]:
df_train, df_val = train_test_split(
    train_data,
    test_size=0.2,
    stratify=train_data["target"],
    shuffle=True,
    random_state=RANDOM_STATE,
)

def print_stats(df: pd.DataFrame):
    num_normal = len(df[df["target"] == "Normal"])
    num_abnormal = len(df[df["target"] == "AbNormal"])

    print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}" + f" ratio: {num_abnormal/num_normal}")


# Print statistics
print(f"  \tAbnormal\tNormal")
print_stats(df_train)
print_stats(df_val)

  	Abnormal	Normal
  Total: Normal: 30524, AbNormal: 1880 ratio: 0.06159087930808544
  Total: Normal: 7632, AbNormal: 470 ratio: 0.061582809224318656


## 3. 모델 학습

### 모델 정의

optuna

In [109]:
import optuna
from lightgbm import LGBMClassifier

# 'Normal'과 'AbNormal'을 숫자로 변환
train_data['target'] = train_data['target'].map({'Normal': 0, 'AbNormal': 1})

# 스레드홀드 설정
THRESHOLD = 0.3

def objectiveLGBM_dart(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 3000),
        'num_leaves': trial.suggest_int('num_leaves', 500, 3000),
        'max_depth': trial.suggest_int('max_depth', 10, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'min_child_samples': trial.suggest_int('min_child_samples', 3, 300),
        
        'boosting_type': 'dart',  # 'boosting'를 'boosting_type'으로 수정
        'random_state': RANDOM_STATE,
        'verbose': -1
    }
       
    model = LGBMClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data.drop("target", axis=1),
    train_data["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveLGBM_dart(trial, x_train, y_train, x_val, y_val), n_trials=300)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


In [110]:
from lightgbm import LGBMClassifier

def create_model(n_estimators, num_leaves, max_depth, learning_rate, min_child_samples):
    return LGBMClassifier(
        n_estimators=n_estimators
        , num_leaves=num_leaves
        , max_depth=max_depth
        , learning_rate=learning_rate
        , min_child_samples=min_child_samples
        , verbose=-1
        , boosting='dart'
        , random_state=RANDOM_STATE
        # , class_weight = {1: 1, 0: 5}
    )

model_Dam = create_model(2748, 657, 256, 0.001043279508273329, 58)
model_AutoClave = create_model(1019, 1513, 115, 0.010329666890588058, 7)
model_Fill1 = create_model(979, 1565, 34, 0.04888906225539191, 36)
model_Fill2 = create_model(1019, 1513, 115, 0.010329666890588058, 7)

## 모델 학습

In [111]:
from sklearn.model_selection import train_test_split

def split_data(data, target, test_size=0.2):
    return train_test_split(data, test_size=test_size, stratify=target, random_state=RANDOM_STATE)

df_train_dam, df_val_dam = split_data(train_data_dam, train_data["target"])
df_train_fill1, df_val_fill1 = split_data(train_data_fill1, train_data["target"])
df_train_fill2, df_val_fill2 = split_data(train_data_fill2, train_data["target"])
df_train_autoclave, df_val_autoclave = split_data(train_data_autoclave, train_data["target"])

공정별로 모델 학습 진행

In [112]:
def preprocess_and_train(df, model):
    df["target"] = df["target"].apply(lambda x: 1 if x == 'AbNormal' else 0)
    train_x = df.drop(columns=["target"])
    train_y = df["target"]
    model.fit(train_x, train_y)

# 각 데이터셋과 모델에 대해 함수 호출
preprocess_and_train(df_train_dam, model_Dam)
preprocess_and_train(df_train_fill1, model_Fill1)
preprocess_and_train(df_train_fill2, model_Fill2)
preprocess_and_train(df_train_autoclave, model_AutoClave)

In [113]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning

# UndefinedMetricWarning 경고 무시
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

def preprocess_and_evaluate(df, model, model_name, threshold=0.3):
    df["target"] = df["target"].apply(lambda x: 1 if x == 'AbNormal' else 0)
    val_x = df.drop(columns=["target"])
    val_y = df["target"]
    
    # 확률 예측
    y_pred_proba = model.predict_proba(val_x)[:, 1]  # 양성 클래스에 대한 확률
    
    # 스레스 홀드 적용하여 이진 예측
    y_pred = (y_pred_proba >= threshold).astype(int)
    
    # 모델 성능 평가
    print(f"Model: {model_name}")
    get_clf_eval(val_y, y_pred)
    print("\n")

# 각 데이터셋과 모델에 대해 함수 호출
preprocess_and_evaluate(df_val_dam, model_Dam, "Dam Model")
preprocess_and_evaluate(df_val_fill1, model_Fill1, "Fill1 Model")
preprocess_and_evaluate(df_val_fill2, model_Fill2, "Fill2 Model")
preprocess_and_evaluate(df_val_autoclave, model_AutoClave, "AutoClave Model")

Model: Dam Model
Confusion Matrix:
 [[7344  288]
 [ 374   96]]
Accuracy: 0.9183
Precision: 0.2500
Recall: 0.2043
F1 Score: 0.2248


Model: Fill1 Model
Confusion Matrix:
 [[7411  221]
 [ 388   82]]
Accuracy: 0.9248
Precision: 0.2706
Recall: 0.1745
F1 Score: 0.2122


Model: Fill2 Model
Confusion Matrix:
 [[7339  293]
 [ 377   93]]
Accuracy: 0.9173
Precision: 0.2409
Recall: 0.1979
F1 Score: 0.2173


Model: AutoClave Model
Confusion Matrix:
 [[7260  372]
 [ 369  101]]
Accuracy: 0.9085
Precision: 0.2135
Recall: 0.2149
F1 Score: 0.2142




분할한 데이터 -> 원래의 데이터(train data)로 학습한 새 모델 생성

In [114]:
# SettingWithCopyWarning 경고 무시
pd.options.mode.chained_assignment = None  # default='warn'

In [115]:
# 모델 학습
preprocess_and_train(train_data_dam, model_Dam)
preprocess_and_train(train_data_fill1, model_Fill1)
preprocess_and_train(train_data_fill2, model_Fill2)
preprocess_and_train(train_data_autoclave, model_AutoClave)

In [116]:
# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)

# 각 공정의 예측 확률 계산
probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
]

In [117]:
# 소프트 보팅: 각 모델의 확률 평균 계산
soft_voting_probs = np.mean(probs, axis=0)

# 최종 예측: 평균 확률에 대해 스레드 홀드 0.23 적용
final_predictions = (soft_voting_probs >= 0.23).astype(int)

# 최종 예측 결과 출력
print(sum(final_predictions))

1058


=== Message ===
작성하신 답안 제출이 완료되었습니다.  
Public Score : 0.22286821705426357  

In [132]:
# 소프트 보팅: 각 모델의 확률 평균 계산
soft_voting_probs = np.mean(probs, axis=0)

# 최종 예측: 평균 확률에 대해 스레드 홀드 0.3 적용
final_predictions = (soft_voting_probs >= 0.3).astype(int)

# 최종 예측 결과 출력
print(sum(final_predictions))

572


=== Message ===  
작성하신 답안 제출이 완료되었습니다.  
Public Score : 0.22835633626097868  

In [133]:
# 소프트 보팅: 각 모델의 확률 평균 계산
soft_voting_probs = np.mean(probs, axis=0)

# 최종 예측: 평균 확률에 대해 스레드 홀드 0.28 적용
final_predictions = (soft_voting_probs >= 0.28).astype(int)

# 최종 예측 결과 출력
print(sum(final_predictions))

678


=== Message ===  
작성하신 답안 제출이 완료되었습니다.  
Public Score : 0.22958579881656804

In [141]:
# 소프트 보팅: 각 모델의 확률 평균 계산
soft_voting_probs = np.mean(probs, axis=0)

# 최종 예측: 평균 확률에 대해 스레드 홀드 0.28 적용
final_predictions = (soft_voting_probs >= 0.26).astype(int)

# 최종 예측 결과 출력
print(sum(final_predictions))

807


=== Message ===    
작성하신 답안 제출이 완료되었습니다.    
Public Score : 0.22417582417582418    

## 4. 제출하기


### 제출 파일 작성


In [142]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

In [143]:
df_sub['target'].value_counts()

Normal      16554
AbNormal      807
Name: target, dtype: int64

In [144]:
df_sub.head(10)

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
5,001fda4596f545d0a3b0ce85fbea77d2,Normal
6,0020734a7b29472298358ad58645a0c9,Normal
7,00234c5914cd4c4a888d13f8b3773135,Normal
8,00297b6c93e44d49ac534758a23dc74e,Normal
9,002d904240d84b188d410d16383a9c3a,Normal


**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
